In [1]:
%%bash
which python
#conda install pytorch torchvision cpuonly -c pytorch

/home/robot/miniconda3/envs/backprop/bin/python


In [1]:
import sys
print(sys.version)
import torch
import torch.nn as nn

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from collections import OrderedDict

# Import PyTorch

from torch.autograd import Variable

from torch.autograd import Function # import Function to create custom activations
from torch.nn.parameter import Parameter # import Parameter to create custom activations with learnable parameters
from torch import optim # import optimizers for demonstrations
import torch.nn.functional as F # import torch functions
from torchvision import datasets, transforms # import transformations to use for demo

3.7.5 (default, Oct 25 2019, 15:51:11) 
[GCC 7.3.0]


In [3]:
def generate_random_goals(numb_agents):
    goals = np.random.rand(numb_agents,2)
    return goals

def generate_radnom_init_poses(numb_agents):
    init_poses = np.random.rand(numb_agents,2)
    return init_poses

In [2]:
weight = 0.01
class Agent():
    def __init__(self):
        self.init_pose = 10*np.random.rand(2)
        self.goal_pose = 10*np.random.rand(2)
        self.cur_pose = self.init_pose
        self.speed_amplitude = 0.05
    
    def get_dist_to(self,pose):
        return np.linalg.norm(self.cur_pose - pose)

    def make_step(self,delta_pose):
        self.cur_pose += delta_pose
    
    def apply_HSFM(self,agents):
        f_rep = np.array([0.,0.])
        for agent in agents:
            # cumulative  distruction force
            f_rep += weight * ((agent.cur_pose - self.cur_pose))

        speed_orient = f_rep + ((self.goal_pose - self.cur_pose))
        self.make_step( speed_orient/np.linalg.norm(speed_orient) * self.speed_amplitude)
    
    def __str__(self):
        return "cur pose: " + str(self.cur_pose) + "  goal_pose: " + str(self.goal_pose) + "  distance to goal: " +\
                    str(self.get_dist_to(self.goal_pose))


agents = [Agent(),Agent(),Agent(),Agent(),Agent(),Agent(),Agent()]
a = Agent()
print (a)
for i in range(0,200):
    a.apply_HSFM(agents)
for i in range (0,10):
    a.apply_HSFM(agents)
    print (a)

for agent in agents:
    print (agent.cur_pose)
        
        

cur pose: [1.26286951 2.71677431]  goal_pose: [7.51093958 7.37547905]  distance to goal: 7.793709609148863
cur pose: [7.25690125 7.30742607]  goal_pose: [7.51093958 7.37547905]  distance to goal: 0.26299559849689624
cur pose: [7.29659682 7.33782773]  goal_pose: [7.51093958 7.37547905]  distance to goal: 0.21762454183023777
cur pose: [7.25690125 7.30742607]  goal_pose: [7.51093958 7.37547905]  distance to goal: 0.26299559849689624
cur pose: [7.29659682 7.33782773]  goal_pose: [7.51093958 7.37547905]  distance to goal: 0.21762454183023777
cur pose: [7.25690125 7.30742607]  goal_pose: [7.51093958 7.37547905]  distance to goal: 0.26299559849689624
cur pose: [7.29659682 7.33782773]  goal_pose: [7.51093958 7.37547905]  distance to goal: 0.21762454183023777
cur pose: [7.25690125 7.30742607]  goal_pose: [7.51093958 7.37547905]  distance to goal: 0.26299559849689624
cur pose: [7.29659682 7.33782773]  goal_pose: [7.51093958 7.37547905]  distance to goal: 0.21762454183023777
cur pose: [7.25690125

In [32]:
class Neural_Network(nn.Module):
    def __init__(self, ):
        super(Neural_Network, self).__init__()
        # parameters
        # TODO: parameters can be parameterized instead of declaring them here
        self.inputSize = 2
        self.outputSize = 1
        self.hiddenSize = 3
        
        # weights
        self.W1 = torch.randn(self.inputSize, self.hiddenSize) # 3 X 2 tensor
        self.W2 = torch.randn(self.hiddenSize, self.outputSize) # 3 X 1 tensor
        
    def forward(self, X):
        self.z = torch.matmul(X, self.W1) # 3 X 3 ".dot" does not broadcast in PyTorch
        print ("len(self.z) ",self.z)
        self.z2 = self.sigmoid(self.z) # activation function 3x3
        self.z3 = torch.matmul(self.z2, self.W2) # 3x1
        o = self.sigmoid(self.z3) # final activation function 3x1
        return o
        
    def sigmoid(self, s):
        
        print (len(s))
        return 1 / (1 + torch.exp(-s))
    
    def sigmoidPrime(self, s):
        # derivative of sigmoid
        return s * (1 - s)
    
    def backward(self, X, y, o):
        self.o_error = y - o # error in output
        self.o_delta = self.o_error * self.sigmoidPrime(o) # derivative of sig to error
        self.z2_error = torch.matmul(self.o_delta, torch.t(self.W2))
        self.z2_delta = self.z2_error * self.sigmoidPrime(self.z2)
        print ("sheet\n",self.z2_delta)
        print ()
        self.W2 += torch.matmul(torch.t(self.z2), self.o_delta)
        self.W1 += torch.matmul(torch.t(X), self.z2_delta)
        
        
        
    def train(self, X, y):
        # forward + backward pass for training
        o = self.forward(X)
        self.backward(X, y, o)
        
    def saveWeights(self, model):
        # we will use the PyTorch internal storage functions
        torch.save(model, "NN")
        # you can reload model with all the weights and so forth with:
        # torch.load("NN")
        
    def predict(self):
        print ("Predicted data based on trained weights: ")
        print ("Input (scaled): \n" + str(xPredicted))
        print ("Output: \n" + str(self.forward(xPredicted)))

In [33]:
X = torch.tensor(([2, 9], [1, 5], [3, 6]), dtype=torch.float) # 3 X 2 tensor
y = torch.tensor(([92], [100], [89]), dtype=torch.float) # 3 X 1 tensor
xPredicted = torch.tensor(([4, 8]), dtype=torch.float) # 1 X 2 tensor
print(X.size())
print(y.size())

# scale units
X_max, _ = torch.max(X, 0)
xPredicted_max, _ = torch.max(xPredicted, 0)

X = torch.div(X, X_max)
xPredicted = torch.div(xPredicted, xPredicted_max)
y = y / 100  # max test score is 100
##################################
print("X = ", X)
print("y = ", y)
##################################

NN = Neural_Network()
for i in range(2000):  # trains the NN 1,000 times
    print ("#" + str(i) + " Loss: " + str(torch.mean((y - NN(X))**2).detach().item()))  # mean sum squared loss
    NN.train(X, y)
NN.saveWeights(NN)
NN.predict()

torch.Size([3, 2])
torch.Size([3, 1])
X =  tensor([[0.6667, 1.0000],
        [0.3333, 0.5556],
        [1.0000, 0.6667]])
y =  tensor([[0.9200],
        [1.0000],
        [0.8900]])
len(self.z)  tensor([[ 0.7335, -0.9674,  1.3846],
        [ 0.3772, -0.5327,  0.7880],
        [ 0.9433, -0.7161,  0.6421]])
3
3
#0 Loss: 0.006895048078149557
len(self.z)  tensor([[ 0.7335, -0.9674,  1.3846],
        [ 0.3772, -0.5327,  0.7880],
        [ 0.9433, -0.7161,  0.6421]])
3
3
sheet
 tensor([[7.2361e-04, 1.6147e-05, 7.5648e-04],
        [4.6363e-03, 1.0975e-04, 5.9064e-03],
        [4.9101e-04, 1.3156e-05, 7.8674e-04]])

len(self.z)  tensor([[ 0.7388, -0.9673,  1.3913],
        [ 0.3801, -0.5327,  0.7916],
        [ 0.9482, -0.7160,  0.6484]])
3
3
#1 Loss: 0.0065581356175243855
len(self.z)  tensor([[ 0.7388, -0.9673,  1.3913],
        [ 0.3801, -0.5327,  0.7916],
        [ 0.9482, -0.7160,  0.6484]])
3
3
sheet
 tensor([[6.5376e-04, 1.8394e-05, 6.8034e-04],
        [4.5177e-03, 1.3465e-04, 5.7368e-

#48 Loss: 0.0030845561996102333
len(self.z)  tensor([[ 0.8390, -0.9595,  1.5090],
        [ 0.4344, -0.5284,  0.8554],
        [ 1.0356, -0.7094,  0.7497]])
3
3
sheet
 tensor([[-2.5844e-04, -3.4506e-05, -2.5194e-04],
        [ 2.5060e-03,  3.4429e-04,  3.0453e-03],
        [-6.0566e-04, -9.7248e-05, -9.4545e-04]])

len(self.z)  tensor([[ 0.8398, -0.9594,  1.5097],
        [ 0.4348, -0.5284,  0.8558],
        [ 1.0362, -0.7093,  0.7501]])
3
3
#49 Loss: 0.003072375198826194
len(self.z)  tensor([[ 0.8398, -0.9594,  1.5097],
        [ 0.4348, -0.5284,  0.8558],
        [ 1.0362, -0.7093,  0.7501]])
3
3
sheet
 tensor([[-2.6326e-04, -3.5420e-05, -2.5658e-04],
        [ 2.4905e-03,  3.4471e-04,  3.0260e-03],
        [-6.1094e-04, -9.8844e-05, -9.5372e-04]])

len(self.z)  tensor([[ 0.8406, -0.9593,  1.5105],
        [ 0.4352, -0.5283,  0.8562],
        [ 1.0367, -0.7093,  0.7505]])
3
3
#50 Loss: 0.003060725284740329
len(self.z)  tensor([[ 0.8406, -0.9593,  1.5105],
        [ 0.4352, -0.5283,  

3
sheet
 tensor([[-3.8732e-04, -6.6424e-05, -3.7887e-04],
        [ 2.0338e-03,  3.5705e-04,  2.4776e-03],
        [-7.4606e-04, -1.5316e-04, -1.1737e-03]])

len(self.z)  tensor([[ 0.8563, -0.9581,  1.5188],
        [ 0.4443, -0.5276,  0.8617],
        [ 1.0419, -0.7099,  0.7442]])
3
3
#102 Loss: 0.002791373757645488
len(self.z)  tensor([[ 0.8563, -0.9581,  1.5188],
        [ 0.4443, -0.5276,  0.8617],
        [ 1.0419, -0.7099,  0.7442]])
3
3
sheet
 tensor([[-3.8844e-04, -6.6817e-05, -3.8007e-04],
        [ 2.0289e-03,  3.5726e-04,  2.4721e-03],
        [-7.4728e-04, -1.5386e-04, -1.1759e-03]])

len(self.z)  tensor([[ 0.8563, -0.9582,  1.5186],
        [ 0.4444, -0.5276,  0.8616],
        [ 1.0417, -0.7100,  0.7437]])
3
3
#103 Loss: 0.0027890929486602545
len(self.z)  tensor([[ 0.8563, -0.9582,  1.5186],
        [ 0.4444, -0.5276,  0.8616],
        [ 1.0417, -0.7100,  0.7437]])
3
3
sheet
 tensor([[-3.8954e-04, -6.7206e-05, -3.8125e-04],
        [ 2.0241e-03,  3.5748e-04,  2.4667e-03],


len(self.z)  tensor([[ 0.8484, -0.9615,  1.4952],
        [ 0.4409, -0.5292,  0.8502],
        [ 1.0229, -0.7155,  0.7043]])
3
3
sheet
 tensor([[-4.2784e-04, -8.4544e-05, -4.2806e-04],
        [ 1.8408e-03,  3.7368e-04,  2.2753e-03],
        [-7.9051e-04, -1.8580e-04, -1.2630e-03]])

len(self.z)  tensor([[ 0.8482, -0.9616,  1.4947],
        [ 0.4407, -0.5292,  0.8499],
        [ 1.0224, -0.7156,  0.7036]])
3
3
#164 Loss: 0.002695644274353981
len(self.z)  tensor([[ 0.8482, -0.9616,  1.4947],
        [ 0.4407, -0.5292,  0.8499],
        [ 1.0224, -0.7156,  0.7036]])
3
3
sheet
 tensor([[-4.2819e-04, -8.4765e-05, -4.2860e-04],
        [ 1.8389e-03,  3.7399e-04,  2.2736e-03],
        [-7.9090e-04, -1.8620e-04, -1.2639e-03]])

len(self.z)  tensor([[ 0.8479, -0.9616,  1.4941],
        [ 0.4406, -0.5293,  0.8496],
        [ 1.0220, -0.7157,  0.7028]])
3
3
#165 Loss: 0.002694477094337344
len(self.z)  tensor([[ 0.8479, -0.9616,  1.4941],
        [ 0.4406, -0.5293,  0.8496],
        [ 1.0220, -0.

len(self.z)  tensor([[ 0.8305, -0.9671,  1.4584],
        [ 0.4320, -0.5320,  0.8316],
        [ 0.9946, -0.7236,  0.6520]])
3
3
#223 Loss: 0.0026310712564736605
len(self.z)  tensor([[ 0.8305, -0.9671,  1.4584],
        [ 0.4320, -0.5320,  0.8316],
        [ 0.9946, -0.7236,  0.6520]])
3
3
sheet
 tensor([[-4.4200e-04, -9.5798e-05, -4.5502e-04],
        [ 1.7593e-03,  3.9447e-04,  2.2140e-03],
        [-8.0588e-04, -2.0644e-04, -1.3089e-03]])

len(self.z)  tensor([[ 0.8302, -0.9672,  1.4577],
        [ 0.4318, -0.5321,  0.8312],
        [ 0.9940, -0.7237,  0.6510]])
3
3
#224 Loss: 0.002630009548738599
len(self.z)  tensor([[ 0.8302, -0.9672,  1.4577],
        [ 0.4318, -0.5321,  0.8312],
        [ 0.9940, -0.7237,  0.6510]])
3
3
sheet
 tensor([[-4.4215e-04, -9.5960e-05, -4.5541e-04],
        [ 1.7584e-03,  3.9484e-04,  2.2135e-03],
        [-8.0603e-04, -2.0673e-04, -1.3095e-03]])

len(self.z)  tensor([[ 0.8298, -0.9673,  1.4570],
        [ 0.4317, -0.5321,  0.8309],
        [ 0.9935, -0

...

len(self.z)  tensor([[ 0.1921, -1.1537,  0.3361],
        [ 0.1030, -0.6268,  0.2522],
        [ 0.1830, -0.9811, -0.7577]])
3
3
#1747 Loss: 0.0011529047042131424
len(self.z)  tensor([[ 0.1921, -1.1537,  0.3361],
        [ 0.1030, -0.6268,  0.2522],
        [ 0.1830, -0.9811, -0.7577]])
3
3
sheet
 tensor([[-0.0006, -0.0003, -0.0010],
        [ 0.0011,  0.0006,  0.0018],
        [-0.0002, -0.0001, -0.0004]])

len(self.z)  tensor([[ 0.1917, -1.1538,  0.3356],
        [ 0.1029, -0.6269,  0.2519],
        [ 0.1826, -0.9812, -0.7583]])
3
3
#1748 Loss: 0.0011524256551638246
len(self.z)  tensor([[ 0.1917, -1.1538,  0.3356],
        [ 0.1029, -0.6269,  0.2519],
        [ 0.1826, -0.9812, -0.7583]])
3
3
sheet
 tensor([[-0.0006, -0.0003, -0.0010],
        [ 0.0011,  0.0006,  0.0018],
        [-0.0002, -0.0001, -0.0004]])

len(self.z)  tensor([[ 0.1914, -1.1539,  0.3351],
        [ 0.1027, -0.6269,  0.2516],
        [ 0.1823, -0.9813, -0.7588]])
3
3
#1749 Loss: 0.0011519416002556682
len(self.z)  

len(self.z)  tensor([[ 0.1738, -1.1576,  0.3090],
        [ 0.0935, -0.6288,  0.2379],
        [ 0.1626, -0.9865, -0.7883]])
3
3
sheet
 tensor([[-0.0006, -0.0003, -0.0010],
        [ 0.0011,  0.0006,  0.0018],
        [-0.0002, -0.0001, -0.0003]])

len(self.z)  tensor([[ 0.1735, -1.1576,  0.3085],
        [ 0.0933, -0.6288,  0.2377],
        [ 0.1622, -0.9866, -0.7888]])
3
3
#1805 Loss: 0.0011257358128204942
len(self.z)  tensor([[ 0.1735, -1.1576,  0.3085],
        [ 0.0933, -0.6288,  0.2377],
        [ 0.1622, -0.9866, -0.7888]])
3
3
sheet
 tensor([[-0.0006, -0.0003, -0.0010],
        [ 0.0011,  0.0006,  0.0018],
        [-0.0002, -0.0001, -0.0003]])

len(self.z)  tensor([[ 0.1732, -1.1577,  0.3080],
        [ 0.0931, -0.6288,  0.2375],
        [ 0.1619, -0.9867, -0.7894]])
3
3
#1806 Loss: 0.0011252821423113346
len(self.z)  tensor([[ 0.1732, -1.1577,  0.3080],
        [ 0.0931, -0.6288,  0.2375],
        [ 0.1619, -0.9867, -0.7894]])
3
3
sheet
 tensor([[-0.0006, -0.0003, -0.0010],
   

len(self.z)  tensor([[ 0.1548, -1.1614,  0.2808],
        [ 0.0835, -0.6307,  0.2231],
        [ 0.1415, -0.9919, -0.8198]])
3
3
#1865 Loss: 0.001099198474548757
len(self.z)  tensor([[ 0.1548, -1.1614,  0.2808],
        [ 0.0835, -0.6307,  0.2231],
        [ 0.1415, -0.9919, -0.8198]])
3
3
sheet
 tensor([[-5.9417e-04, -2.7155e-04, -1.0051e-03],
        [ 1.0345e-03,  5.8779e-04,  1.7552e-03],
        [-1.8954e-04, -9.4059e-05, -2.7740e-04]])

len(self.z)  tensor([[ 0.1545, -1.1615,  0.2803],
        [ 0.0833, -0.6307,  0.2229],
        [ 0.1412, -0.9919, -0.8203]])
3
3
#1866 Loss: 0.0010987682035192847
len(self.z)  tensor([[ 0.1545, -1.1615,  0.2803],
        [ 0.0833, -0.6307,  0.2229],
        [ 0.1412, -0.9919, -0.8203]])
3
3
sheet
 tensor([[-5.9427e-04, -2.7163e-04, -1.0053e-03],
        [ 1.0342e-03,  5.8773e-04,  1.7547e-03],
        [-1.8915e-04, -9.3879e-05, -2.7676e-04]])

len(self.z)  tensor([[ 0.1542, -1.1615,  0.2798],
        [ 0.0832, -0.6308,  0.2226],
        [ 0.1408, 

#1923 Loss: 0.0010749181965366006
len(self.z)  tensor([[ 0.1373, -1.1648,  0.2546],
        [ 0.0742, -0.6324,  0.2093],
        [ 0.1223, -0.9966, -0.8486]])
3
3
sheet
 tensor([[-5.9975e-04, -2.7632e-04, -1.0156e-03],
        [ 1.0177e-03,  5.8416e-04,  1.7266e-03],
        [-1.6732e-04, -8.3673e-05, -2.4154e-04]])

len(self.z)  tensor([[ 0.1370, -1.1648,  0.2541],
        [ 0.0741, -0.6325,  0.2091],
        [ 0.1219, -0.9967, -0.8491]])
3
3
#1924 Loss: 0.0010745105100795627
len(self.z)  tensor([[ 0.1370, -1.1648,  0.2541],
        [ 0.0741, -0.6325,  0.2091],
        [ 0.1219, -0.9967, -0.8491]])
3
3
sheet
 tensor([[-5.9984e-04, -2.7640e-04, -1.0158e-03],
        [ 1.0174e-03,  5.8409e-04,  1.7261e-03],
        [-1.6695e-04, -8.3497e-05, -2.4094e-04]])

len(self.z)  tensor([[ 0.1367, -1.1649,  0.2537],
        [ 0.0739, -0.6325,  0.2089],
        [ 0.1216, -0.9967, -0.8496]])
3
3
#1925 Loss: 0.0010741016594693065
len(self.z)  tensor([[ 0.1367, -1.1649,  0.2537],
        [ 0.0739, -0

3
sheet
 tensor([[-6.0535e-04, -2.8118e-04, -1.0258e-03],
        [ 1.0009e-03,  5.8040e-04,  1.6976e-03],
        [-1.4501e-04, -7.3083e-05, -2.0633e-04]])

len(self.z)  tensor([[ 0.1191, -1.1681,  0.2272],
        [ 0.0646, -0.6341,  0.1949],
        [ 0.1025, -1.0012, -0.8783]])
3
3
#1985 Loss: 0.001050311722792685
len(self.z)  tensor([[ 0.1191, -1.1681,  0.2272],
        [ 0.0646, -0.6341,  0.1949],
        [ 0.1025, -1.0012, -0.8783]])
3
3
sheet
 tensor([[-6.0544e-04, -2.8125e-04, -1.0260e-03],
        [ 1.0006e-03,  5.8034e-04,  1.6971e-03],
        [-1.4465e-04, -7.2910e-05, -2.0576e-04]])

len(self.z)  tensor([[ 0.1188, -1.1682,  0.2268],
        [ 0.0645, -0.6342,  0.1947],
        [ 0.1022, -1.0013, -0.8788]])
3
3
#1986 Loss: 0.0010499239433556795
len(self.z)  tensor([[ 0.1188, -1.1682,  0.2268],
        [ 0.0645, -0.6342,  0.1947],
        [ 0.1022, -1.0013, -0.8788]])
3
3
sheet
 tensor([[-6.0553e-04, -2.8133e-04, -1.0261e-03],
        [ 1.0003e-03,  5.8028e-04,  1.6967e-03]

In [24]:
def silu(input):
    '''
    Applies the Sigmoid Linear Unit (SiLU) function element-wise:
        SiLU(x) = x * sigmoid(x)
    '''
    return input * torch.sigmoid(input) # use torch.sigmoid to make sure that we created the most efficient implemetation based on builtin PyTorch functions

# create a class wrapper from PyTorch nn.Module, so
# the function now can be easily used in models
class SiLU(nn.Module):
    '''
    Applies the Sigmoid Linear Unit (SiLU) function element-wise:
        SiLU(x) = x * sigmoid(x)
    Shape:
        - Input: (N, *) where * means, any number of additional
          dimensions
        - Output: (N, *), same shape as the input
    References:
        -  Related paper:
        https://arxiv.org/pdf/1606.08415.pdf
    Examples:
        >>> m = silu()
        >>> input = torch.randn(2)
        >>> output = m(input)
    '''
    def __init__(self):
        '''
        Init method.
        '''
        super().__init__() # init the base class

    def forward(self, input):
        '''
        Forward pass of the function.
        '''
        return silu(input) # simply apply already implemented SiLU

In [25]:
# helper function to train a model
def train_model(model, trainloader):
    '''
    Function trains the model and prints out the training log.
    INPUT:
        model - initialized PyTorch model ready for training.
        trainloader - PyTorch dataloader for training data.
    '''
    #setup training

    #define loss function
    criterion = nn.NLLLoss()
    #define learning rate
    learning_rate = 0.003
    #define number of epochs
    epochs = 5
    #initialize optimizer
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    #run training and print out the loss to make sure that we are actually fitting to the training set
    print('Training the model. Make sure that loss decreases after each epoch.\n')
    for e in range(epochs):
        running_loss = 0
        for images, labels in trainloader:
            images = images.view(images.shape[0], -1)
            log_ps = model(images)
            loss = criterion(log_ps, labels)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
        else:
            # print out the loss to make sure it is decreasing
            print(f"Training loss: {running_loss}")

In [26]:
activation_function = SiLU()

# Initialize the model using nn.Sequential
model = nn.Sequential(OrderedDict([
                      ('fc1', nn.Linear(784, 256)),
                      ('activation1', activation_function), # use SiLU
                      ('fc2', nn.Linear(256, 128)),
                      ('bn2', nn.BatchNorm1d(num_features=128)),
                      ('activation2', activation_function), # use SiLU
                      ('dropout', nn.Dropout(0.3)),
                      ('fc3', nn.Linear(128, 64)),
                      ('bn3', nn.BatchNorm1d(num_features=64)),
                      ('activation3', activation_function), # use SiLU
                      ('logits', nn.Linear(64, 10)),
                      ('logsoftmax', nn.LogSoftmax(dim=1))]))

# Run training
# train_model(model)

In [30]:
lin = nn.Linear(5, 5)  # maps from R^5 to R^3, parameters A, b
# data is 2x5.  A maps from 5 to 3... can we map "data" under A?
data = torch.randn(5, 5)
print("data\n",data)
print ("after activ\n", activation_function(data))
print(lin(data))  # yes
# dir(lin)
# lin.weight

data
 tensor([[-1.4413, -1.5798,  0.2201, -1.9385, -0.8376],
        [ 0.1204,  2.1931,  0.0313,  0.8220, -1.9160],
        [ 0.4480, -0.9469,  1.7359,  0.2348, -0.7383],
        [-0.9024, -0.6932,  0.8006,  0.7885, -0.6257],
        [-0.5239, -0.6185,  0.7787, -0.3742, -0.8152]])
after activ
 tensor([[-0.2758, -0.2699,  0.1221, -0.2439, -0.2530],
        [ 0.0638,  1.9730,  0.0159,  0.5710, -0.2458],
        [ 0.2733, -0.2647,  1.4758,  0.1311, -0.2387],
        [-0.2604, -0.2311,  0.5525,  0.5421, -0.2180],
        [-0.1949, -0.2165,  0.5337, -0.1525, -0.2501]])
tensor([[ 2.0696, -1.2290,  0.4429, -0.7002, -1.1235],
        [-0.4790, -0.0472,  1.8795,  1.4659,  0.7191],
        [ 0.8990, -1.0779,  0.2113,  0.6360, -0.0779],
        [ 0.5753, -0.7269,  0.5934,  0.6678,  0.1212],
        [ 1.0104, -0.8614,  0.5872,  0.2600, -0.3237]],
       grad_fn=<AddmmBackward>)


In [61]:
class LinearFunction(Function):

    # Note that both forward and backward are @staticmethods
    @staticmethod
    # bias is an optional argument
    def forward(ctx, input, weight, bias=None):
        ctx.save_for_backward(input, weight, bias)
        output = input.mm(weight.t())
        if bias is not None:
            output += bias.unsqueeze(0).expand_as(output)
        return output

    # This function has only a single output, so it gets only one gradient
    @staticmethod
    def backward(ctx, grad_output):
        # This is a pattern that is very convenient - at the top of backward
        # unpack saved_tensors and initialize all gradients w.r.t. inputs to
        # None. Thanks to the fact that additional trailing Nones are
        # ignored, the return statement is simple even when the function has
        # optional inputs.
        input, weight, bias = ctx.saved_tensors
        grad_input = grad_weight = grad_bias = None

        # These needs_input_grad checks are optional and there only to
        # improve efficiency. If you want to make your code simpler, you can
        # skip them. Returning gradients for inputs that don't require it is
        # not an error.
        if ctx.needs_input_grad[0]:
            grad_input = grad_output.mm(weight)
        if ctx.needs_input_grad[1]:
            grad_weight = grad_output.t().mm(input)
        if bias is not None and ctx.needs_input_grad[2]: 
            grad_bias = grad_output.sum(0).squeeze(0)

        return grad_input, grad_weight, grad_bias

    
class Linear(nn.Module):
    def __init__(self, input_features, output_features, bias=True):
        super(Linear, self).__init__()
        self.input_features = input_features
        self.output_features = output_features

        # nn.Parameter is a special kind of Tensor, that will get
        # automatically registered as Module's parameter once it's assigned
        # as an attribute. Parameters and buffers need to be registered, or
        # they won't appear in .parameters() (doesn't apply to buffers), and
        # won't be converted when e.g. .cuda() is called. You can use
        # .register_buffer() to register buffers.
        # nn.Parameters require gradients by default.
        self.weight = nn.Parameter(torch.Tensor(output_features, input_features))
        if bias:
            self.bias = nn.Parameter(torch.Tensor(output_features))
        else:
            # You should always register all possible parameters, but the
            # optional ones can be None if you want.
            self.register_parameter('bias', None)

        # Not a very smart way to initialize weights
        self.weight.data.uniform_(-0.1, 0.1)
        if bias is not None:
            self.bias.data.uniform_(-0.1, 0.1)

    def forward(self, input):
        # See the autograd section for explanation of what happens here.
        print (input[1][1:3])
        return LinearFunction.apply(input, self.weight, self.bias)

    def extra_repr(self):
        # (Optional)Set the extra information about this module. You can test
        # it by printing an object of this class.
        return 'in_features={}, out_features={}, bias={}'.format(
            self.in_features, self.out_features, self.bias is not None
        )

l  = Linear(5,5)
data = torch.randn(5, 5)
print (data)
l(data)

tensor([[ 1.7270,  0.0559,  0.3615,  1.5843, -0.2128],
        [ 0.3252,  1.6818,  0.3243, -0.7382, -1.1336],
        [ 0.7501, -0.0913, -0.6908, -0.5237, -0.2603],
        [ 0.8230,  0.2017, -0.3591,  0.4825, -0.5546],
        [-0.6115,  0.5781,  0.8511, -0.8328,  0.1706]])
tensor([1.6818, 0.3243])


tensor([[ 0.0525, -0.1868, -0.1970, -0.1350,  0.1735],
        [ 0.1642,  0.0180, -0.0429, -0.1444, -0.0956],
        [ 0.0860,  0.0794,  0.0429, -0.0227,  0.0516],
        [ 0.0446, -0.0195, -0.0395, -0.1149,  0.0715],
        [ 0.0835,  0.1354,  0.0857, -0.0018, -0.0588]],
       grad_fn=<LinearFunctionBackward>)

In [76]:
x = Variable(torch.ones(2, 2) * 2, requires_grad=True)
z = 2 * (x * x) + 5 * x
print ("z=\n",z,"\n")
print (type(z))


z.backward(torch.ones(2, 2))
print("x.grad ", x.grad)

z=
 tensor([[18., 18.],
        [18., 18.]], grad_fn=<AddBackward0>) 

<class 'torch.Tensor'>
x.grad  tensor([[13., 13.],
        [13., 13.]])


In [56]:
import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F


In [117]:
class Net(nn.Module):

    def __init__(self, num_agents = 10):
        super(Net, self).__init__()

        # an affine operation: y = Wx + b
        self.fc1 = nn.Linear(num_agents, num_agents)
        self.fc2 = nn.Linear(num_agents, num_agents)
        self.fc3 = nn.Linear(num_agents, num_agents)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x



net = Net()
print(net)

Net(
  (fc1): Linear(in_features=10, out_features=10, bias=True)
  (fc2): Linear(in_features=10, out_features=10, bias=True)
  (fc3): Linear(in_features=10, out_features=10, bias=True)
)


In [123]:
# check number of trainable parameters
def print_numb_train_params(net):
    model_parameters = filter(lambda p: p.requires_grad, net.parameters())
    params = sum([np.prod(p.size()) for p in model_parameters])
    print ("model has ",params, " trainable parameters")

In [148]:
# test net 
net = Net()
input = Variable(torch.randn(1, 10))
print_numb_train_params(net)
out = net(input)
print (out)



model has  330  trainable parameters
tensor([[-0.4255, -0.2568, -0.1426, -0.0648,  0.1813, -0.3765, -0.2103, -0.2545,
          0.2611, -0.2381]], grad_fn=<AddmmBackward>)


In [149]:
params = list(net.parameters())
for index,p in enumerate(params):
    if index == 0 :
        continue
    p.requires_grad = False
    
    
print_numb_train_params(net)

model has  100  trainable parameters


In [150]:
# x =  torch.tensor(([1.,2.,3],[1.,2.,0]))
# print (x)
# print (x.size())
# print (x.size()[0])
# linear = nn.Linear(3,3)
# print (linear(x))


In [162]:
output = net(input)
target = Variable(torch.randn(1, 10))   # Create a dummy true label Size 10.
criterion = nn.MSELoss()
target = target.view(1, 10)
# Compute the loss by MSE of the output and the true label
print (output.size())
print (target.size())
loss = criterion(output, target)         # Size 1

net.zero_grad()      # zeroes the gradient buffers of all parameters

loss.backward()

# Print the gradient for the bias parameters of the first convolution layer
print(net.fc1.weight.grad)

torch.Size([1, 10])
torch.Size([1, 10])
tensor([[ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000],
        [-0.0050,  0.0111,  0.0190,  0.0181,  0.0322,  0.0019, -0.0027, -0.0193,
         -0.0027,  0.0126],
        [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000],
        [ 0.0008, -0.0017, -0.0029, -0.0028, -0.0050, -0.0003,  0.0004,  0.0030,
          0.0004, -0.0019],
        [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000],
        [ 0.0231, -0.0507, -0.0870, -0.0828, -0.1478, -0.0089,  0.0126,  0.0887,
          0.0125, -0.0578],
        [-0.0047,  0.0102,  0.0176,  0.0167,  0.0299,  0.0018, -0.0025, -0.0179,
         -0.0025,  0.0117],
        [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
       

In [164]:
import torch.optim as optim

# Create a optimizer
learning_rate = 1e-4
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# Inside the training loop
for t in range(500):
    output = net(input)
    loss = criterion(output, target)

    optimizer.zero_grad()   # zero the gradient buffers
    loss.backward()

    optimizer.step()        # Perform the training parameters update

In [166]:
import random
import torch
from torch.autograd import Variable


class DynamicNet(torch.nn.Module):
    def __init__(self, D_in, H, D_out):
        super(DynamicNet, self).__init__()
        self.input_linear = torch.nn.Linear(D_in, H)
        self.middle_linear = torch.nn.Linear(H, H)
        self.output_linear = torch.nn.Linear(H, D_out)

    def forward(self, x):
        h_relu = self.input_linear(x).clamp(min=0)
        for _ in range(random.randint(0, 3)):
            h_relu = self.middle_linear(h_relu).clamp(min=0)
        y_pred = self.output_linear(h_relu)
        return y_pred


N, D_in, H, D_out = 64, 1000, 100, 10

x = Variable(torch.randn(N, D_in))
y = Variable(torch.randn(N, D_out), requires_grad=False)
print (y)
model = DynamicNet(D_in, H, D_out)

criterion = torch.nn.MSELoss(size_average=False)
optimizer = torch.optim.SGD(model.parameters(), lr=1e-4, momentum=0.9)
for t in range(5000):
    y_pred = model(x)
    loss = criterion(y_pred, y)
    print (loss)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
# print (y_pred)

tensor([[ 1.3027e-01,  1.8761e+00,  1.0362e+00, -2.0249e+00,  2.7885e-01,
         -4.4408e-01,  1.9033e-01, -3.1425e-03,  8.6965e-01, -2.7640e-02],
        [ 4.8088e-01, -3.3056e-01,  5.4735e-01, -5.1855e-01,  7.3330e-01,
          1.4976e+00, -6.8621e-01,  8.1652e-01, -3.8945e-01,  1.6182e+00],
        [ 2.5351e-01, -1.1422e+00, -1.6019e+00,  1.5334e+00, -5.7351e-01,
         -8.5846e-01,  1.0687e-01, -2.4757e-01,  1.0819e+00, -6.0554e-01],
        [-1.9184e+00,  4.0474e-01, -7.5887e-01,  8.1002e-01, -6.7193e-01,
          7.6462e-01, -6.5722e-01, -6.8494e-01,  9.5387e-01, -1.8925e+00],
        [-4.8331e-01,  7.3129e-01,  2.7746e-01, -1.1787e+00, -2.1424e-01,
         -4.7897e-01,  2.8844e-01,  6.2623e-01,  9.1693e-01,  3.6361e-01],
        [-1.0339e+00,  1.0644e+00,  5.7459e-01,  1.9826e+00, -8.0765e-01,
          2.3773e-01, -1.7418e+00, -8.4978e-01,  2.3145e-02, -8.3303e-02],
        [ 1.6593e+00,  6.0489e-01,  6.1794e-01, -2.2564e-01, -4.0033e-02,
         -3.7358e-01,  2.0825e+0

tensor(206.9187, grad_fn=<MseLossBackward>)
tensor(501.1830, grad_fn=<MseLossBackward>)
tensor(327.2002, grad_fn=<MseLossBackward>)
tensor(284.8870, grad_fn=<MseLossBackward>)
tensor(248.5770, grad_fn=<MseLossBackward>)
tensor(614.9540, grad_fn=<MseLossBackward>)
tensor(278.8681, grad_fn=<MseLossBackward>)
tensor(370.1485, grad_fn=<MseLossBackward>)
tensor(237.8034, grad_fn=<MseLossBackward>)
tensor(168.5560, grad_fn=<MseLossBackward>)
tensor(168.4700, grad_fn=<MseLossBackward>)
tensor(230.1522, grad_fn=<MseLossBackward>)
tensor(122.1780, grad_fn=<MseLossBackward>)
tensor(189.7579, grad_fn=<MseLossBackward>)
tensor(150.6481, grad_fn=<MseLossBackward>)
tensor(179.2309, grad_fn=<MseLossBackward>)
tensor(142.2396, grad_fn=<MseLossBackward>)
tensor(82.7815, grad_fn=<MseLossBackward>)
tensor(161.7030, grad_fn=<MseLossBackward>)
tensor(102.0197, grad_fn=<MseLossBackward>)
tensor(73.4396, grad_fn=<MseLossBackward>)
tensor(109.6760, grad_fn=<MseLossBackward>)
tensor(87.6125, grad_fn=<MseLossBa

tensor(11.0330, grad_fn=<MseLossBackward>)
tensor(7.4533, grad_fn=<MseLossBackward>)
tensor(7.8454, grad_fn=<MseLossBackward>)
tensor(13.9884, grad_fn=<MseLossBackward>)
tensor(25.7156, grad_fn=<MseLossBackward>)
tensor(43.1772, grad_fn=<MseLossBackward>)
tensor(10.1850, grad_fn=<MseLossBackward>)
tensor(4.4976, grad_fn=<MseLossBackward>)
tensor(12.1949, grad_fn=<MseLossBackward>)
tensor(11.0558, grad_fn=<MseLossBackward>)
tensor(2.5994, grad_fn=<MseLossBackward>)
tensor(16.5937, grad_fn=<MseLossBackward>)
tensor(2.3390, grad_fn=<MseLossBackward>)
tensor(31.7711, grad_fn=<MseLossBackward>)
tensor(4.1977, grad_fn=<MseLossBackward>)
tensor(4.9254, grad_fn=<MseLossBackward>)
tensor(4.0482, grad_fn=<MseLossBackward>)
tensor(10.9939, grad_fn=<MseLossBackward>)
tensor(13.1368, grad_fn=<MseLossBackward>)
tensor(10.8747, grad_fn=<MseLossBackward>)
tensor(10.7369, grad_fn=<MseLossBackward>)
tensor(2.8311, grad_fn=<MseLossBackward>)
tensor(2.7202, grad_fn=<MseLossBackward>)
tensor(4.7335, grad_f

tensor(0.4963, grad_fn=<MseLossBackward>)
tensor(1.1697, grad_fn=<MseLossBackward>)
tensor(0.6596, grad_fn=<MseLossBackward>)
tensor(0.9228, grad_fn=<MseLossBackward>)
tensor(0.8011, grad_fn=<MseLossBackward>)
tensor(0.5583, grad_fn=<MseLossBackward>)
tensor(0.7337, grad_fn=<MseLossBackward>)
tensor(0.6586, grad_fn=<MseLossBackward>)
tensor(0.5985, grad_fn=<MseLossBackward>)
tensor(0.4939, grad_fn=<MseLossBackward>)
tensor(0.4002, grad_fn=<MseLossBackward>)
tensor(0.6893, grad_fn=<MseLossBackward>)
tensor(0.9099, grad_fn=<MseLossBackward>)
tensor(0.3315, grad_fn=<MseLossBackward>)
tensor(0.9474, grad_fn=<MseLossBackward>)
tensor(0.3099, grad_fn=<MseLossBackward>)
tensor(0.8902, grad_fn=<MseLossBackward>)
tensor(0.7841, grad_fn=<MseLossBackward>)
tensor(0.7272, grad_fn=<MseLossBackward>)
tensor(0.6622, grad_fn=<MseLossBackward>)
tensor(0.3845, grad_fn=<MseLossBackward>)
tensor(0.4983, grad_fn=<MseLossBackward>)
tensor(0.4668, grad_fn=<MseLossBackward>)
tensor(0.3784, grad_fn=<MseLossBac

tensor(0.0972, grad_fn=<MseLossBackward>)
tensor(0.0930, grad_fn=<MseLossBackward>)
tensor(0.0819, grad_fn=<MseLossBackward>)
tensor(0.1368, grad_fn=<MseLossBackward>)
tensor(0.0545, grad_fn=<MseLossBackward>)
tensor(0.4480, grad_fn=<MseLossBackward>)
tensor(0.0524, grad_fn=<MseLossBackward>)
tensor(0.3874, grad_fn=<MseLossBackward>)
tensor(0.3699, grad_fn=<MseLossBackward>)
tensor(0.4859, grad_fn=<MseLossBackward>)
tensor(0.1135, grad_fn=<MseLossBackward>)
tensor(0.4688, grad_fn=<MseLossBackward>)
tensor(0.0762, grad_fn=<MseLossBackward>)
tensor(0.0607, grad_fn=<MseLossBackward>)
tensor(0.3902, grad_fn=<MseLossBackward>)
tensor(0.3667, grad_fn=<MseLossBackward>)
tensor(0.2062, grad_fn=<MseLossBackward>)
tensor(0.0454, grad_fn=<MseLossBackward>)
tensor(0.4398, grad_fn=<MseLossBackward>)
tensor(0.3281, grad_fn=<MseLossBackward>)
tensor(0.3474, grad_fn=<MseLossBackward>)
tensor(0.2075, grad_fn=<MseLossBackward>)
tensor(0.2205, grad_fn=<MseLossBackward>)
tensor(0.4930, grad_fn=<MseLossBac

...

tensor(0.0030, grad_fn=<MseLossBackward>)
tensor(0.0118, grad_fn=<MseLossBackward>)
tensor(0.0133, grad_fn=<MseLossBackward>)
tensor(0.0130, grad_fn=<MseLossBackward>)
tensor(0.0020, grad_fn=<MseLossBackward>)
tensor(0.0124, grad_fn=<MseLossBackward>)
tensor(0.0009, grad_fn=<MseLossBackward>)
tensor(0.0108, grad_fn=<MseLossBackward>)
tensor(0.0098, grad_fn=<MseLossBackward>)
tensor(0.0014, grad_fn=<MseLossBackward>)
tensor(0.0016, grad_fn=<MseLossBackward>)
tensor(0.0205, grad_fn=<MseLossBackward>)
tensor(0.0198, grad_fn=<MseLossBackward>)
tensor(0.0010, grad_fn=<MseLossBackward>)
tensor(0.0162, grad_fn=<MseLossBackward>)
tensor(0.0027, grad_fn=<MseLossBackward>)
tensor(0.0128, grad_fn=<MseLossBackward>)
tensor(0.0153, grad_fn=<MseLossBackward>)
tensor(0.0157, grad_fn=<MseLossBackward>)
tensor(0.0151, grad_fn=<MseLossBackward>)
tensor(0.0034, grad_fn=<MseLossBackward>)
tensor(0.0130, grad_fn=<MseLossBackward>)
tensor(0.0027, grad_fn=<MseLossBackward>)
tensor(0.0111, grad_fn=<MseLossBac

tensor(0.0016, grad_fn=<MseLossBackward>)
tensor(0.0069, grad_fn=<MseLossBackward>)
tensor(0.0061, grad_fn=<MseLossBackward>)
tensor(0.0193, grad_fn=<MseLossBackward>)
tensor(0.0055, grad_fn=<MseLossBackward>)
tensor(0.0051, grad_fn=<MseLossBackward>)
tensor(0.0009, grad_fn=<MseLossBackward>)
tensor(0.0009, grad_fn=<MseLossBackward>)
tensor(0.0017, grad_fn=<MseLossBackward>)
tensor(0.0214, grad_fn=<MseLossBackward>)
tensor(0.0200, grad_fn=<MseLossBackward>)
tensor(0.0055, grad_fn=<MseLossBackward>)
tensor(0.0019, grad_fn=<MseLossBackward>)
tensor(0.0069, grad_fn=<MseLossBackward>)
tensor(0.0158, grad_fn=<MseLossBackward>)
tensor(0.0073, grad_fn=<MseLossBackward>)
tensor(0.0143, grad_fn=<MseLossBackward>)
tensor(0.0025, grad_fn=<MseLossBackward>)
tensor(0.0027, grad_fn=<MseLossBackward>)
tensor(0.0010, grad_fn=<MseLossBackward>)
tensor(0.0110, grad_fn=<MseLossBackward>)
tensor(0.0007, grad_fn=<MseLossBackward>)
tensor(0.0144, grad_fn=<MseLossBackward>)
tensor(0.0019, grad_fn=<MseLossBac

tensor(0.0024, grad_fn=<MseLossBackward>)
tensor(0.0427, grad_fn=<MseLossBackward>)
tensor(0.0069, grad_fn=<MseLossBackward>)
tensor(0.0053, grad_fn=<MseLossBackward>)
tensor(0.0294, grad_fn=<MseLossBackward>)
tensor(0.0007, grad_fn=<MseLossBackward>)
tensor(0.0156, grad_fn=<MseLossBackward>)
tensor(0.0116, grad_fn=<MseLossBackward>)
tensor(0.0108, grad_fn=<MseLossBackward>)
tensor(0.0153, grad_fn=<MseLossBackward>)
tensor(0.0052, grad_fn=<MseLossBackward>)
tensor(0.0116, grad_fn=<MseLossBackward>)
tensor(0.0038, grad_fn=<MseLossBackward>)
tensor(0.0042, grad_fn=<MseLossBackward>)
tensor(0.0401, grad_fn=<MseLossBackward>)
tensor(0.0029, grad_fn=<MseLossBackward>)
tensor(0.0227, grad_fn=<MseLossBackward>)
tensor(0.0030, grad_fn=<MseLossBackward>)
tensor(0.0061, grad_fn=<MseLossBackward>)
tensor(0.0296, grad_fn=<MseLossBackward>)
tensor(0.0033, grad_fn=<MseLossBackward>)
tensor(0.0020, grad_fn=<MseLossBackward>)
tensor(0.0152, grad_fn=<MseLossBackward>)
tensor(0.0016, grad_fn=<MseLossBac

tensor(0.0068, grad_fn=<MseLossBackward>)
tensor(0.0021, grad_fn=<MseLossBackward>)
tensor(0.0199, grad_fn=<MseLossBackward>)
tensor(0.0018, grad_fn=<MseLossBackward>)
tensor(0.0055, grad_fn=<MseLossBackward>)
tensor(0.0048, grad_fn=<MseLossBackward>)
tensor(0.0015, grad_fn=<MseLossBackward>)
tensor(0.0191, grad_fn=<MseLossBackward>)
tensor(0.0024, grad_fn=<MseLossBackward>)
tensor(0.0057, grad_fn=<MseLossBackward>)
tensor(0.0156, grad_fn=<MseLossBackward>)
tensor(0.0145, grad_fn=<MseLossBackward>)
tensor(0.0085, grad_fn=<MseLossBackward>)
tensor(0.0024, grad_fn=<MseLossBackward>)
tensor(0.0013, grad_fn=<MseLossBackward>)
tensor(0.0022, grad_fn=<MseLossBackward>)
tensor(0.0022, grad_fn=<MseLossBackward>)
tensor(0.0008, grad_fn=<MseLossBackward>)
tensor(0.0019, grad_fn=<MseLossBackward>)
tensor(0.0017, grad_fn=<MseLossBackward>)
tensor(0.0006, grad_fn=<MseLossBackward>)
tensor(0.0006, grad_fn=<MseLossBackward>)
tensor(0.0085, grad_fn=<MseLossBackward>)
tensor(0.0065, grad_fn=<MseLossBac

tensor(0.0046, grad_fn=<MseLossBackward>)
tensor(0.0155, grad_fn=<MseLossBackward>)
tensor(0.0038, grad_fn=<MseLossBackward>)
tensor(0.0151, grad_fn=<MseLossBackward>)
tensor(0.0043, grad_fn=<MseLossBackward>)


# force calc V1

In [239]:
state = torch.tensor(([150.,150.],[1.,2.],[2.,0.], [5.,5.]))


s1 = state - state[0] # find [x1-xi,y1-y1; x2-xi,y2-yi;...]
# print (s1)
s2 = s1**2 # [(x1-xi)**2, (y1-y1)**2 ; (x2-xi)**2, (y2-yi)**2;...]
# print (s2)
s3 = s2[:,0] + s2[:,1] # [(x1-xi)**2+(y1-y1)**2 ; (x2-xi)**2+(y2-yi)**2;...]]
# print (s3)
s4 = torch.sqrt(s3) #[d0,d1,d2,d3...]
# print (s4)
s5 = 1/s4 # [1/d0, 1/d1,....]
# print ("s5", s5)
s55 = s1/s4.view(4,1)
s55[s55 != s55] = 0
s5[s5 == float('inf')] = 0
s5.matmul(s55)


tensor([-0.0102, -0.0102])

# Force calc V2

In [214]:
def tensor_inf_to_zero(bad_tensor):
    bad_tensor[bad_tensor == float('inf')] = 0
    return bad_tensor


# TODO: tests
def calc_forces(state, prints = False):
    state = state.view(-1,2)
    if (prints):
        print ("    --- state\n",state)
    state_concated = state
    state_concated_t = state.view(1,-1)
    for i in range(0,state.shape[0]-1):
        state_concated = torch.cat((state,state_concated),dim=1)
        state_concated_t = torch.cat([state_concated_t,state.view(1,-1)])
    
    if (prints):
        print ("    ---st1\n",state_concated.size(), "\n", state_concated)
        
    
    delta_pose = state_concated_t - state_concated
    if (prints):
        print ("    ---delta_pose\n",delta_pose.size(), "\n", delta_pose)
    
    auxullary = torch.zeros(state_concated.shape[0], state_concated.shape[1])
    for i in range(state_concated.shape[0]):

        auxullary[i,2*i] = 1.
        auxullary[i,2*i+1] = 1.

    if (prints):
        print("    ----auxullary\n",auxullary)
    outputs = auxullary*delta_pose
    s_d = torch.sum(outputs, dim=0)

    st2 = (delta_pose - s_d)**2
    if (prints):
        print ("    ----distance squred.shape\n", st2.shape, "\n", st2)
    aux = auxullary.t()
    st3 = torch.sqrt(st2.matmul(aux)) ## aka distance
    if (prints):
        print ("    ----dist\n", st3)
    
    force_amplitude = tensor_inf_to_zero(1/st3) ## aka 1/distance
    
    if (prints):
        print ("    ----st4(force_amplitude)\n ", force_amplitude)

    force = force_amplitude.matmul(auxullary) * delta_pose/st3.matmul(auxullary)
    force[force != force] = 0
    if (prints):
        print ("    ----force(before nan hanlding)\n ", force)
    aux1 = torch.tensor(([1.,0.],[0.,1.]))
    for i in range(0,state.shape[0]-1):
        aux1 = torch.cat((aux1,torch.tensor(([1.,0.],[0.,1.]))))
    force = force.matmul(aux1)
    
    return force

## Let's measure calc time 

In [232]:
from timeit import default_timer as timer
state = torch.tensor(([0.,0.],[2.,1.],[-1,-1],[1.,2.], [10, 10], [0.2,0.5]))

times = []
for i in range (0,1000):
    start = timer()
    calc_forces(state)
    end = timer()
    times.append(end - start)
#     print (times[-1])
print ("average time: ", sum(times)/(float(len(times))))

state = torch.tensor(([0.,0.],[2.,1.],[-1,-1],[1.,2.], [10, 10], [0.2,0.5],[0.,0.],[2.,1.],[-1,-1],[1.,2.], [10, 10], [0.2,0.5]))
times = []
for i in range (0,1000):
    start = timer()
    t = calc_forces(state)
    end = timer()
    times.append(end - start)
#     print (times[-1])
print ("average time: ", sum(times)/(float(len(times))))
# print (t)

average time:  0.00045260089066869114
average time:  0.0007193408769671805


In [219]:
# state = torch.tensor(([0.,0.],[2.,1.],[-1,-1],[1.,2.], [10, 10], [0.2,0.5]))
state = torch.tensor(([0.,0.],[2.,1.],[-1,-1]))
calc_forces(state)



tensor([[-0.1000, -0.3000],
        [-0.6308, -0.3538],
        [ 0.7308,  0.6538]])

In [217]:
test = torch.tensor([[0.5, 1.3, 0., 0.],
        [2., 1., 2., 1.]])

test

tensor([[0.5000, 1.3000, 0.0000, 0.0000],
        [2.0000, 1.0000, 2.0000, 1.0000]])

In [156]:
goal = torch.tensor([4.0,1.0])
current_pose = torch.tensor([1.0,1.0])
desired_velocity = torch.tensor([10.0,10.0])
sfm_params = {"fg":0.1}
def force_goal(current_pose ,goal, desired_velocity, sfm_params):
    v_desiredxy = (goal - current_pose)
    v_desired = torch.sqrt(torch.sum(torch.pow(v_desiredxy,2)))
    v_desiredxy = torch.mul(v_desiredxy,desired_velocity)/v_desired
    fg = torch.tensor([sfm_params['fg']*(v_desiredxy[0]-current_pose[0]*torch.cos(current_pose[1])),sfm_params['fg']*(v_desiredxy[0]-current_pose[0]*torch.sin(current_pose[1]))])
    # fg = [0.1,0.1]
    return fg
print(force_goal(current_pose ,goal,desired_velocity,sfm_params))

tensor([0.9460, 0.9159])


In [202]:
import torch
x = torch.ones(2, 2, requires_grad=True)
print(x)
y = x + 2
print(y)
print(y.grad_fn)

z = y * y * 3
print (z)
z.backward(x)
# out = z.mean()

# print(z, out)
# print (out.backward())
#torch.ones(2, 2)
print(x.grad)


tensor([[1., 1.],
        [1., 1.]], requires_grad=True)
tensor([[3., 3.],
        [3., 3.]], grad_fn=<AddBackward0>)
tensor([[27., 27.],
        [27., 27.]], grad_fn=<MulBackward0>)
tensor([[18., 18.],
        [18., 18.]])


In [233]:
a = torch.randn(2, 2)
a = ((a * 3) / (a - 1))
print(a.requires_grad)
a.requires_grad_(True)
print(a.requires_grad)
b = (a * a).sum()
print(b.grad_fn)



False
True


In [181]:
x = torch.randn(3, requires_grad=True)

y = x * 2
while y.data.norm() < 1000:
    y = y * 2

print(y)
print(y.data.norm())
v = torch.tensor([0.1, 1.0, 0.0001], dtype=torch.float)
y.backward(v)

print(x.grad)

tensor([1221.1653, -439.9754,  624.2367], grad_fn=<MulBackward0>)
tensor(1440.3105)
tensor([1.0240e+02, 1.0240e+03, 1.0240e-01])


In [239]:
import torch

print("Trial 1: with python float")
w = torch.randn(3,5,requires_grad = True) * 0.01

x = torch.randn(5,4,requires_grad = True)

y = torch.matmul(w,x).sum(1)
w.retain_grad()
y.backward(torch.ones(3))

print ("y = ", y)
print("w.requires_grad:",w.requires_grad)
print("x.requires_grad:",x.requires_grad)

print("w.grad",w.grad)
print("x.grad",x.grad)



print("Trial 2: with on-the-go torch scalar")
w = torch.randn(3,5,requires_grad = True) * torch.tensor(0.01,requires_grad=True)

x = torch.randn(5,4,requires_grad = True)

y = torch.matmul(w,x).sum(1)

y.backward(torch.ones(3))

print("w.requires_grad:",w.requires_grad)
print("x.requires_grad:",x.requires_grad)

print("w.grad",w.grad)
print("x.grad",x.grad)

print("Trial 3: with named torch scalar")
t = torch.tensor(0.01,requires_grad=True)
w = torch.randn(3,5,requires_grad = True) * t

x = torch.randn(5,4,requires_grad = True)

y = torch.matmul(w,x).sum(1)

y.backward(torch.ones(3))

print("w.requires_grad:",w.requires_grad)
print("x.requires_grad:",x.requires_grad)

print("w.grad",w.grad)
print("x.grad",x.grad)

Trial 1: with python float
y =  tensor([-0.0296,  0.0068,  0.0306], grad_fn=<SumBackward1>)
w.requires_grad: True
x.requires_grad: True
w.grad tensor([[ 2.3965, -0.5110,  0.0040,  0.1300,  1.3100],
        [ 2.3965, -0.5110,  0.0040,  0.1300,  1.3100],
        [ 2.3965, -0.5110,  0.0040,  0.1300,  1.3100]])
x.grad tensor([[-0.0226, -0.0226, -0.0226, -0.0226],
        [-0.0258, -0.0258, -0.0258, -0.0258],
        [ 0.0095,  0.0095,  0.0095,  0.0095],
        [ 0.0288,  0.0288,  0.0288,  0.0288],
        [ 0.0344,  0.0344,  0.0344,  0.0344]])
Trial 2: with on-the-go torch scalar
w.requires_grad: True
x.requires_grad: True
w.grad None
x.grad tensor([[-0.0028, -0.0028, -0.0028, -0.0028],
        [ 0.0085,  0.0085,  0.0085,  0.0085],
        [-0.0065, -0.0065, -0.0065, -0.0065],
        [-0.0023, -0.0023, -0.0023, -0.0023],
        [ 0.0076,  0.0076,  0.0076,  0.0076]])
Trial 3: with named torch scalar
w.requires_grad: True
x.requires_grad: True
w.grad None
x.grad tensor([[0.0168, 0.0168, 0

In [260]:
xx = Variable(torch.ones(2,2), requires_grad = True)
yy = 3*xx

zz = (yy**2  * torch.tensor(([1.,0.],[5,7]))).matmul(torch.tensor(([-1.,-10.],[5,7])))
zz = zz.matmul(torch.tensor(([1.,0.,3],[5,7,0])))
zz = zz.matmul(torch.tensor(([1.,0.],[5,7],[1.,2])))
print (zz)
zz.backward(torch.ones(2,2))
print(xx.grad) # This is ok


tensor([[-3636., -4464.],
        [  720.,  1179.]], grad_fn=<MmBackward>)
tensor([[-16200.,      0.],
        [-81000.,  84798.]])
